In [ ]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.worksheet.cell_range import CellRange
from openpyxl.formula.translate import Translator
import copy


def build_ordered_block_map(wb_path, key_sheet_name, key_col, value_col, header_order_range):
    """Reads headers/subheaders from Excel, builds ordered dict."""
    wb = load_workbook(wb_path, data_only=True)
    ws = wb[key_sheet_name]

    # Read ordered headers list
    ordered_headers = [
        cell.value for cell in ws[header_order_range]
        if cell.value is not None
    ]

    # Build raw pairs
    raw_pairs = []
    for row in ws.iter_rows(min_row=2, max_col=ws.max_column):
        header = row[column_index_from_string(key_col) - 1].value
        subheader = row[column_index_from_string(value_col) - 1].value
        if header and subheader:
            raw_pairs.append((header.strip(), subheader.strip()))

    # Build ordered map
    block_map = {}
    for key in ordered_headers:
        block_map[key] = []

    for header, subheader in raw_pairs:
        if header == subheader:
            continue  # skip if same, we treat it as header-only
        if header in block_map and subheader not in block_map[header]:
            block_map[header].append(subheader)

    wb.close()
    return block_map


def replicate_blocks_with_tracking(
    file_path,
    sheet_name,
    source_block_range,
    block_map,
    output_start_col,
    header_row=3,
    text_row=5
):
    wb = load_workbook(file_path)
    ws = wb[sheet_name]

    start_col_idx = column_index_from_string(source_block_range[0])
    end_col_idx = column_index_from_string(source_block_range[1])
    num_cols_to_copy = end_col_idx - start_col_idx + 1

    current_col_idx = output_start_col

    block_tracker = []

    for header, subheaders in block_map.items():
        # 1️⃣ — replicate subheader blocks first
        for subheader in subheaders:
            coords = paste_block(ws, start_col_idx, end_col_idx, current_col_idx,
                                 header_row, text_row, header, subheader)
            block_tracker.append({
                'type': 'subheader',
                'header': header,
                'name': subheader,
                'cols': coords
            })
            current_col_idx += num_cols_to_copy + 1  # gap

        # 2️⃣ — replicate header-only block
        coords = paste_block(ws, start_col_idx, end_col_idx, current_col_idx,
                             header_row, text_row, header, header)
        block_tracker.append({
            'type': 'header',
            'header': header,
            'name': header,
            'cols': coords
        })
        current_col_idx += num_cols_to_copy + 1

    wb.save(file_path)
    wb.close()

    print("\n✅✅ All blocks created and tracked.\n")
    for entry in block_tracker:
        print(entry)
    return block_tracker


def paste_block(ws, src_start_idx, src_end_idx, dest_start_idx, header_row, text_row, header, subheader):
    num_cols = src_end_idx - src_start_idx + 1

    for row in ws.iter_rows(
        min_col=src_start_idx,
        max_col=src_end_idx,
        min_row=1,
        max_row=ws.max_row
    ):
        for i, source_cell in enumerate(row):
            dest_col_idx = dest_start_idx + i
            new_cell = ws.cell(row=source_cell.row, column=dest_col_idx)

            # Formula handling
            if source_cell.data_type == 'f':
                orig_formula = source_cell.value
                translated = Translator(orig_formula, origin=source_cell.coordinate).translate_formula(new_cell.coordinate)
                new_cell.value = translated
            else:
                new_cell.value = source_cell.value

            new_cell.font = copy.copy(source_cell.font)
            new_cell.border = copy.copy(source_cell.border)
            new_cell.fill = copy.copy(source_cell.fill)
            new_cell.number_format = copy.copy(source_cell.number_format)
            new_cell.protection = copy.copy(source_cell.protection)
            new_cell.alignment = copy.copy(source_cell.alignment)

    # Merged header row
    merged_start = get_column_letter(dest_start_idx)
    merged_end = get_column_letter(dest_start_idx + num_cols - 1)
    merge_range = f"{merged_start}{header_row}:{merged_end}{header_row}"
    ws.merge_cells(merge_range)
    ws[f"{merged_start}{header_row}"].value = header

    # Merged text row
    merge_range_text = f"{merged_start}{text_row}:{merged_end}{text_row}"
    ws.merge_cells(merge_range_text)
    ws[f"{merged_start}{text_row}"].value = subheader

    return f"{merged_start}:{merged_end}"


# ==========================
# ✅ ✅ ✅ EXAMPLE USAGE
# ==========================

if __name__ == "__main__":
    # 1️⃣ Build block map from key sheet
    block_map = build_ordered_block_map(
        wb_path="your_dashboard.xlsx",
        key_sheet_name="Keys",
        key_col="A",
        value_col="B",
        header_order_range="D1:D10"  # For example — adjust to your file
    )

    # 2️⃣ Paste blocks with tracking
    tracker = replicate_blocks_with_tracking(
        file_path="your_dashboard.xlsx",
        sheet_name="Dashboard",
        source_block_range=("V", "AC"),
        block_map=block_map,
        output_start_col=column_index_from_string("AE"),
        header_row=3,
        text_row=5
    )


In [ ]:
import xlwings as xw

def build_dashboard_with_check_blocks(
    file_path,
    template_block_range,
    check_block_range,
    output_sheet_name,
    headers_order,
    subheaders_dict,
    start_col
):
    app = xw.App(visible=True)
    wb = app.books.open(file_path)
    ws = wb.sheets[output_sheet_name]

    # Where to start pasting blocks
    col_ptr = start_col

    for header in headers_order:
        subheaders = subheaders_dict.get(header, [])

        # If header has subheaders, paste subheader blocks first
        for subheader in subheaders:
            template_range = ws.range(template_block_range)
            template_range.copy(ws.range((template_range.row, col_ptr)))
            print(f"Pasted subheader block '{subheader}' at column {col_ptr}")
            col_ptr += template_range.columns.count + 1  # +1 for spacing

        # Then paste header block
        template_range = ws.range(template_block_range)
        template_range.copy(ws.range((template_range.row, col_ptr)))
        print(f"Pasted header block '{header}' at column {col_ptr}")
        col_ptr += template_range.columns.count + 1

    # When all header & subheader blocks are done, paste the Check block
    check_template = ws.range(check_block_range)
    check_template.copy(ws.range((check_template.row, col_ptr)))
    print(f"Pasted Check block at column {col_ptr}")

    col_ptr += check_template.columns.count + 1

    # Finally: delete the **original template check block**
    check_template.columns.delete()
    print("Deleted original Check block template columns.")

    # Save changes
    wb.save()
    wb.close()
    app.quit()
    print(f"✅ Dashboard built & saved: {file_path}")


# Example usage:
if __name__ == "__main__":
    headers_order = ["A", "B"]
    subheaders_dict = {
        "A": ["A1", "A2"],
        "B": ["B1"]
    }

    build_dashboard_with_check_blocks(
        file_path="your_dashboard.xlsx",
        template_block_range="V3:AC20",  # Example block
        check_block_range="AE3:AL20",    # Example Check block range
        output_sheet_name="Dashboard",
        headers_order=headers_order,
        subheaders_dict=subheaders_dict,
        start_col=22  # Col V is col 22
    )


In [ ]:
from openpyxl import load_workbook

def fix_product_control_bfg(file_path):
    wb = load_workbook(file_path)
    ws = wb["Data"]

    # Assume header is in row 2
    headers = [cell.value.strip() if cell.value else None for cell in ws[2]]

    try:
        bf_idx = headers.index("Business Framework") + 1  # openpyxl is 1-based
        bfg_idx = headers.index("Business Framework Group") + 1
    except ValueError:
        print("Required columns not found.")
        return

    updated_count = 0

    for row in ws.iter_rows(min_row=3, max_row=ws.max_row):
        bf_value = row[bf_idx - 1].value
        if bf_value and bf_value.strip().upper() == "PRODUCT CONTROL":
            bfg_cell = row[bfg_idx - 1]
            if bfg_cell.value != "L3 - Financial Control & Tax":
                bfg_cell.value = "L3 - Financial Control & Tax"
                updated_count += 1

    wb.save(file_path)
    wb.close()

    print(f"✅ Done. Updated {updated_count} rows.")

# Example usage:
fix_product_control_bfg("YourReport.xlsx")


In [ ]:
import win32com.client as win32

def update_product_control(filepath):
    # Start Excel
    excel = win32.gencache.EnsureDispatch("Excel.Application")
    excel.Visible = False  # True for debugging

    wb = excel.Workbooks.Open(filepath)
    ws = wb.Sheets("Data")

    # Find last used row
    last_row = ws.Cells(ws.Rows.Count, 1).End(-4162).Row  # xlUp

    # Find columns
    headers = [ws.Cells(2, col).Value for col in range(1, ws.UsedRange.Columns.Count + 1)]
    bf_col = headers.index("Business Framework") + 1
    bfg_col = headers.index("Business Framework Group") + 1

    # Apply AutoFilter on Business Framework
    ws.Range(ws.Cells(2, bf_col), ws.Cells(last_row, bfg_col)).AutoFilter(Field=bf_col, Criteria1="PRODUCT CONTROL")

    # Get visible rows in filter (skip header)
    visible = ws.Range(ws.Cells(3, bf_col), ws.Cells(last_row, bf_col)).SpecialCells(12)  # 12 = xlCellTypeVisible
    rows = sorted([cell.Row for cell in visible])

    if not rows:
        print("✅ No PRODUCT CONTROL rows found. Nothing to do.")
    else:
        # Find continuous blocks for fast bulk update
        blocks = []
        block = [rows[0]]
        for r1, r2 in zip(rows, rows[1:]):
            if r2 == r1 + 1:
                block.append(r2)
            else:
                blocks.append(block)
                block = [r2]
        blocks.append(block)

        for block in blocks:
            first, last = block[0], block[-1]
            length = last - first + 1
            ws.Range(ws.Cells(first, bfg_col), ws.Cells(last, bfg_col)).Value = [["L3 - Financial Control & Tax"]] * length

        print(f"✅ Updated PRODUCT CONTROL for {len(rows)} rows in {len(blocks)} blocks.")

    # Clear filter
    ws.AutoFilterMode = False

    wb.Save()
    wb.Close(False)
    excel.Quit()

if __name__ == "__main__":
    update_product_control(r"C:\Path\To\Your\Report.xlsx")


In [ ]:
import xlwings as xw
import os
import shutil

def run_task_two():
    # === CONFIG ===
    master_file = "Base_Dashboard.xlsx"
    report_names = [
        "Investor Relations",
        "Stress Testing",
        "APEX (FC)",
        "Finance COO",
        "Financial Support"
    ]

    sheets_and_cells = [
        ("Index", "D3"),
        ("Summary", "D3"),
        ("P&L Item View", "B3"),
        ("DC by Country", "B3"),
        ("Headcount by Country", "B3"),
        ("Driller", "B3")
    ]

    with xw.App(visible=False) as app:
        for report_name in report_names:
            # 📌 Open a fresh copy each loop
            wb = app.books.open(master_file)
            
            # Get month from 'Dates'!C4
            month_text = wb.sheets['Dates'].range('C4').value  # e.g. 'May-25'
            month_part, yy_part = month_text.split('-')
            year_full = "20" + yy_part
            month_year = f"{month_part} {year_full}"  # 'May 2025'

            # Find Reporting sheet
            reporting_sheet = None
            for sh in wb.sheets:
                if 'Reporting' in sh.name:
                    reporting_sheet = sh
                    break

            if not reporting_sheet:
                raise ValueError("No sheet with 'Reporting' found.")

            # Update Reporting sheet name
            new_reporting_name = f"{month_year} Reporting"
            reporting_sheet.name = new_reporting_name

            # Update Reporting sheet cells
            reporting_ws = wb.sheets[new_reporting_name]
            reporting_ws.range("C20").value = report_name
            reporting_ws.range("B25").value = f"{month_year} Results"

            # Update other sheets/cells
            for sheet_name, cell in sheets_and_cells:
                ws = wb.sheets[sheet_name]
                ws.range(cell).value = report_name

            # Save
            output_name = f"Dashboard_{report_name}.xlsx"
            wb.save(os.path.join(os.getcwd(), output_name))
            wb.close()

        print("✅ All reports created!")

if __name__ == "__main__":
    run_task_two()


In [1]:
#Working fine for duplicating cuntries block with formula updation, but not handling blockof totals.- Task one

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.worksheet.cell_range import CellRange
from openpyxl.formula.translate import Translator
import copy


def replicate_dashboard_block_with_translated_formulas(
    file_path,
    sheet_name,
    source_start_col,
    source_end_col,
    country_list,
    header_row=3,
    text_row=5
):
    wb = load_workbook(file_path)
    ws = wb[sheet_name]

    start_col_idx = column_index_from_string(source_start_col)
    end_col_idx = column_index_from_string(source_end_col)
    num_cols_to_copy = end_col_idx - start_col_idx + 1

    # Where to start inserting the first block (+2 means 1 blank col gap)
    current_col_idx = end_col_idx + 2

    for country in country_list:
        print(f"🔹 Adding block for: {country}")

        # --- Copy the block ---
        for row in ws.iter_rows(
            min_col=start_col_idx,
            max_col=end_col_idx,
            min_row=1,
            max_row=ws.max_row
        ):
            for i, source_cell in enumerate(row):
                new_col_idx = current_col_idx + i
                new_cell = ws.cell(row=source_cell.row, column=new_col_idx)

                # ✅ Copy value or translated formula
                if source_cell.data_type == 'f':
                    orig_formula = source_cell.value
                    origin = source_cell.coordinate
                    target = new_cell.coordinate
                    new_formula = Translator(orig_formula, origin=origin).translate_formula(target)
                    new_cell.value = new_formula
                else:
                    new_cell.value = source_cell.value

                # ✅ Copy style
                new_cell.font = copy.copy(source_cell.font)
                new_cell.border = copy.copy(source_cell.border)
                new_cell.fill = copy.copy(source_cell.fill)
                new_cell.number_format = copy.copy(source_cell.number_format)
                new_cell.protection = copy.copy(source_cell.protection)
                new_cell.alignment = copy.copy(source_cell.alignment)

        # --- Handle merged header row ---
        merged_start = get_column_letter(current_col_idx)
        merged_end = get_column_letter(current_col_idx + num_cols_to_copy - 1)
        new_merge_range = f"{merged_start}{header_row}:{merged_end}{header_row}"

        # Remove overlaps if needed
        for merge in list(ws.merged_cells.ranges):
            if CellRange(new_merge_range).coord in merge.coord:
                ws.merged_cells.ranges.remove(merge)

        ws.merge_cells(new_merge_range)
        ws[f"{merged_start}{header_row}"] = country  # ✅ Only top-left cell gets value

        # Copy header style
        source_header_cell = ws[f"{source_start_col}{header_row}"]
        new_header_cell = ws[f"{merged_start}{header_row}"]
        new_header_cell.font = copy.copy(source_header_cell.font)
        new_header_cell.border = copy.copy(source_header_cell.border)
        new_header_cell.fill = copy.copy(source_header_cell.fill)
        new_header_cell.number_format = copy.copy(source_header_cell.number_format)
        new_header_cell.protection = copy.copy(source_header_cell.protection)
        new_header_cell.alignment = copy.copy(source_header_cell.alignment)

        # --- Fill ROW 4 below merged header with same country name ---
        for i in range(num_cols_to_copy):
            col_letter = get_column_letter(current_col_idx + i)
            cell = ws[f"{col_letter}{header_row + 1}"]
            cell.value = country

            # Optional: match style from the same row in the source block
            source_cell = ws[f"{get_column_letter(start_col_idx + i)}{header_row + 1}"]
            cell.font = copy.copy(source_cell.font)
            cell.border = copy.copy(source_cell.border)
            cell.fill = copy.copy(source_cell.fill)
            cell.number_format = copy.copy(source_cell.number_format)
            cell.protection = copy.copy(source_cell.protection)
            cell.alignment = copy.copy(source_cell.alignment)

        # --- Handle merged text row ---
        text_merge_start = get_column_letter(current_col_idx)
        text_merge_end = get_column_letter(current_col_idx + (column_index_from_string("Y") - start_col_idx))
        new_text_merge_range = f"{text_merge_start}{text_row}:{text_merge_end}{text_row}"

        for merge in list(ws.merged_cells.ranges):
            if CellRange(new_text_merge_range).coord in merge.coord:
                ws.merged_cells.ranges.remove(merge)

        ws.merge_cells(new_text_merge_range)
        ws[f"{text_merge_start}{text_row}"] = ws[f"{source_start_col}{text_row}"].value

        text_source_cell = ws[f"{source_start_col}{text_row}"]
        text_new_cell = ws[f"{text_merge_start}{text_row}"]
        text_new_cell.font = copy.copy(text_source_cell.font)
        text_new_cell.border = copy.copy(text_source_cell.border)
        text_new_cell.fill = copy.copy(text_source_cell.fill)
        text_new_cell.number_format = copy.copy(text_source_cell.number_format)
        text_new_cell.protection = copy.copy(text_source_cell.protection)
        text_new_cell.alignment = copy.copy(text_source_cell.alignment)

        # Advance for next block (+1 for blank col)
        current_col_idx += num_cols_to_copy + 1

    wb.save(file_path)
    wb.close()
    print("✅✅ All blocks copied successfully with translated formulas, merged headers, and country name in row 4!")


# --- EXAMPLE ---
if __name__ == "__main__":
    replicate_dashboard_block_with_translated_formulas(
        file_path="your_dashboard.xlsx",
        sheet_name="Dashboard",
        source_start_col="V",
        source_end_col="AC",
        country_list=["Paris", "Hongkong", "India"],
        header_row=3,
        text_row=5
    )


Processing... /

KeyboardInterrupt: 

In [ ]:
import os
import pandas as pd

from openpyxl import load_workbook

# Set folder path where your Excel files are stored
folder_path = r'path_to_your_folder'  # <-- Replace this with your actual folder path

# To store flagged files and sheet names
issues = []

# Loop through each Excel file in the folder
for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file)
        try:
            wb = load_workbook(file_path, data_only=True)
            sheet_names = wb.sheetnames

            # Check only sheet 2 and 3 (index 1 and 2)
            for sheet_index in [1, 2]:
                if sheet_index < len(sheet_names):
                    ws = wb[sheet_names[sheet_index]]

                    for row in ws.iter_rows(min_row=1, values_only=True):
                        if row[2] == 'Check':  # Column C is index 2 (0-based)
                            # From Column E (index 4) onwards
                            values_to_check = row[4:]
                            if any(val != 0 and val is not None for val in values_to_check):
                                issues.append((file, sheet_names[sheet_index]))
                            break  # No need to search further in this sheet
        except Exception as e:
            print(f"Error processing {file}: {e}")

# Print the issues
if issues:
    print("❌ Files with failed validation:")
    for file_name, sheet_name in issues:
        print(f"File: {file_name}, Sheet: {sheet_name}")
else:
    print("✅ All files passed the validation checks.")


In [4]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox

# Required sheet and column mapping
required_sheets = {
    'RTN': 'RTN',
    'Location': 'Country',
    'GCB': 'GCB'
}

def select_workbook():
    """Prompts the user to select an Excel file."""
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    file_path = filedialog.askopenfilename(
        title="Select Excel Workbook",
        filetypes=[("Excel files", "*.xlsx")]
    )
    if not file_path:
        raise Exception("No file selected.")
    return file_path

def validate_workbook(file_path):
    """Validates presence of required sheets and columns."""
    try:
        xl = pd.ExcelFile(file_path)
    except Exception as e:
        raise Exception(f"Failed to open Excel file: {e}")

    for sheet, required_col in required_sheets.items():
        if sheet not in xl.sheet_names:
            raise Exception(f"Missing required sheet: '{sheet}'")

        df = xl.parse(sheet)
        df.columns = df.columns.str.strip()  # Remove extra spaces

        if required_col not in df.columns:
            raise Exception(f"Sheet '{sheet}' must contain column '{required_col}'")
        
        if df[required_col].isnull().any():
            raise Exception(f"Column '{required_col}' in sheet '{sheet}' contains blank values")

    print("Workbook validation successful.")

if __name__ == "__main__":
    try:
        file_path = select_workbook()
        validate_workbook(file_path)
        print(f"Processing file: {file_path}")
        # Continue with your processing logic here...
    except Exception as e:
        print(f"Error: {e}")
